In [49]:
import pandas as pd
import ast
#reading the data   
lines_path = "1/movie_lines.txt"

lines_df = pd.read_csv(
    lines_path,
    sep=" \+\+\+\$\+\+\+ ",
    engine='python',
    header=None,
    names=['lineID', 'characterID', 'movieID', 'characterName', 'text'],
    encoding='latin-1'  # <-- Important
)
conv_path = "1/movie_conversations.txt"

conv_df = pd.read_csv(
    conv_path,
    sep=" \+\+\+\$\+\+\+ ",
    engine='python',
    header=None,
    names=['character1', 'character2', 'movieID', 'lineIDs'],
    encoding='latin-1'
)

# Convert the string representation of list to an actual Python list
conv_df['lineIDs'] = conv_df['lineIDs'].apply(ast.literal_eval)
id2line = {}
for inbdx,row in lines_df.iterrows():
    id2line[row["lineID"]] = row['text']


<>:8: SyntaxWarning: invalid escape sequence '\+'
<>:18: SyntaxWarning: invalid escape sequence '\+'
<>:8: SyntaxWarning: invalid escape sequence '\+'
<>:18: SyntaxWarning: invalid escape sequence '\+'
C:\Users\ahmed\AppData\Local\Temp\ipykernel_23624\373611191.py:8: SyntaxWarning: invalid escape sequence '\+'
  sep=" \+\+\+\$\+\+\+ ",
C:\Users\ahmed\AppData\Local\Temp\ipykernel_23624\373611191.py:18: SyntaxWarning: invalid escape sequence '\+'
  sep=" \+\+\+\$\+\+\+ ",


In [50]:
#making the input and output pair
pairs = []
for inbdx,row in conv_df.iterrows():
    for i in range(len(row["lineIDs"]) - 1):
        input_lineID  = row["lineIDs"][i]
        output_lineID = row["lineIDs"][i + 1]
        pairs.append((input_lineID,output_lineID))


        

In [51]:
text_pairs = []
for (line_input,line_output) in pairs:
    line_input = id2line[line_input]
    line_output = id2line[line_output]
    text_pairs.append((line_input,line_output))

In [52]:
len(text_pairs)

221616

In [53]:
import numpy as np

arr = np.array(text_pairs,dtype=str)

In [54]:
text_pair = np.char.lower(arr)

In [55]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [56]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [57]:
stop_words = set(stopwords.words('english'))
filterd_text_pair = []
for (i,o) in text_pair:
    i.encode("ascii", "ignore").decode()
    o.encode("ascii", "ignore").decode()
    i = word_tokenize(i)
    o = word_tokenize(o)
    filtered_i = [word for word in i if word not in stop_words]
    filtered_o = [word for word in o if word not in stop_words]
    filterd_text_pair.append((filtered_i,filtered_o))

In [58]:
filtered_text_pair_no_punct = []

for (i, o) in filterd_text_pair:
    # Remove punctuation from input
    filtered_i = [word for word in i if word not in string.punctuation]
    
    # Remove punctuation from output
    filtered_o = [word for word in o if word not in string.punctuation]
    
    filtered_text_pair_no_punct.append((filtered_i, filtered_o))


In [59]:
from collections import Counter
word_freq = Counter()
for (sentence_i,sentence_o) in filtered_text_pair_no_punct:
    word_freq.update(sentence_i)
    word_freq.update(sentence_o)

print(word_freq)

Counter({"'s": 95327, "n't": 81059, '...': 71101, '--': 38787, "'m": 33077, 'know': 32117, "'re": 31277, 'like': 21776, "'ll": 20411, 'get': 19941, 'got': 18951, 'want': 16295, 'think': 15767, "'ve": 15525, 'one': 15130, 'well': 14476, 'right': 14123, 'go': 13923, 'going': 12526, 'would': 12290, 'see': 11579, 'yes': 11482, 'oh': 11281, "'d": 10747, 'yeah': 10671, 'could': 10586, 'good': 10301, 'tell': 9931, 'ca': 9647, 'time': 9454, 'come': 9367, 'say': 9014, 'back': 8690, 'let': 8371, 'man': 8337, 'look': 8218, 'us': 8103, 'never': 8007, 'something': 7975, 'take': 7929, 'mean': 7812, 'na': 7508, "''": 7298, '``': 7250, 'way': 7089, 'really': 6819, 'make': 6772, 'little': 6447, 'sure': 6443, 'okay': 6275, 'gon': 6112, 'people': 6112, 'said': 6017, 'thing': 5943, 'maybe': 5784, 'need': 5766, 'much': 5332, 'two': 5184, 'anything': 5060, 'u': 5044, '/u': 5025, 'give': 4942, 'even': 4761, 'mr.': 4740, 'nothing': 4736, 'talk': 4681, 'sorry': 4634, 'love': 4605, 'thought': 4585, 'ever': 4271

In [60]:
PAD = 0
SOS = 1
EOS = 2
UNK = 3


In [61]:
vocab = {"<PAD>":PAD,
         "<SOS>": SOS
         ,"<EOS>":EOS,
         "<UNK>" :UNK}

In [62]:
i = 4
for word, count in word_freq.most_common():
    if word not in vocab:
        vocab[word] = i 
    i = i + 1


In [63]:
encoded_pairs = []
for (i,o) in filtered_text_pair_no_punct:
    encoded_input = [vocab.get(w, vocab["<UNK>"]) for w in i]
    encoded_output = [vocab["<SOS>"]] + [vocab.get(w, vocab["<UNK>"]) for w in o] + [vocab["<EOS>"]]
    encoded_pairs.append((encoded_input, encoded_output))

In [64]:
from torch.nn.utils.rnn import pad_sequence
import torch
#turn the list into a tensor
filtered_pairs = []
for inp, out in encoded_pairs:
    if len(inp) > 0 and len(out) > 0:
        filtered_pairs.append((inp, out))

input_tensors = [torch.tensor(inp,dtype=torch.long)for inp, out in filtered_pairs]
output_tensors = [torch.tensor(out,dtype=torch.long)for inp, out in filtered_pairs]
#padding them
pad_input = pad_sequence(input_tensors,True,vocab["<PAD>"])
pad_output = pad_sequence(output_tensors,True,vocab["<PAD>"])

In [65]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [66]:
from torch import nn

In [77]:
max_len = max(pad_output,key=len)

In [78]:
print(len(max_len))

294


In [69]:
print(len(pad_input))
print(len(pad_output))

212479
212479


In [80]:
from torch.cuda.amp import GradScaler, autocast

scaler = GradScaler()

C:\Users\ahmed\AppData\Local\Temp\ipykernel_23624\1134786560.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [81]:
class transformer_model(nn.Module):
    def __init__(self, embed_size,vocab_size,num_heads=4,num_layers=3,forward_expansion=4,max_len=165,dropout=0.1):
        super().__init__()
        self.embed_size = embed_size
        self.word_embedding = nn.Embedding(vocab_size,embed_size)
        self.pos_embedding = nn.Embedding(max_len,embed_size)
        decode_layer = nn.TransformerEncoderLayer(d_model=embed_size, nhead=num_heads,
                                                   dim_feedforward=embed_size *forward_expansion, dropout=dropout,batch_first=True)
        self.transformer = nn.TransformerEncoder(decode_layer,num_layers=num_layers)
        
        self.linear = nn.Linear(embed_size,vocab_size)
        self.max_len = max_len
    def forward(self,tgt,padding_mask):
        seq_len = tgt.shape[1]
        position = torch.arange(0,seq_len).unsqueeze(0).to(device)
        tgt = self.word_embedding(tgt) + self.pos_embedding(position)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(seq_len).to(device)
        out = self.transformer(tgt,mask=tgt_mask,
            src_key_padding_mask=padding_mask)
        out = self.linear(out)
        return out 
   

In [82]:
embed_size = 512
dec = transformer_model(embed_size,len(vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=vocab["<PAD>"])
optimizer = torch.optim.Adam(list(dec.parameters()), lr=1e-4)

In [83]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(pad_input, pad_output)
loader = DataLoader(dataset, batch_size=8, shuffle=True)


In [86]:
import tqdm

In [ ]:
def train(dec,device,dataloader,optimizer,pad_indx,loss_fn,epochs = 50):
    dec.train()
    dec.to(device)
    for epoch in range(epochs):
        total_loss = 0
        loop = tqdm(loader, desc=f"Epoch [{epoch+1}/{epochs}]")
        for input , target in dataloader:
            input = input.to(device)
            target = target.to(device)
            input = input[:,:-1]
            target = target[:,1:]
            padding_mask = (input == pad_indx)
            logits = dec(input,padding_mask)
            target = target[:, :logits.size(1)]
            loss = loss_fn(logits.reshape(-1,logits.size(-1)),
                           target.reshape(-1))
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())
        print(f"Epoch {epoch+1}: loss = {total_loss / len(dataloader):.4f}")


In [85]:
train(dec,device,loader,optimizer,vocab["<PAD>"],criterion)

c:\Users\ahmed\OneDrive\Desktop\code\chat bot transformar\.venv\Lib\site-packages\torch\nn\functional.py:6044: UserWarning: Support for mismatched src_key_padding_mask and mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1
Epoch 1


KeyboardInterrupt: 